$$\newcommand{\braket}[3]{\left\langle{#1}\middle|{#2}|{#3}\right\rangle}$$

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$

## Example Notebook for the Trapped Bosons

This Notebook is based on the following [paper](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.114.080402) from the Jochim group.

In [17]:
import pennylane as qml
from pennylane import numpy as np

In [18]:
from pennylane_ls import *
from heroku_credentials import username, password

In [19]:
FermionDevice = qml.device("synqs.fs",shots=500, username = username, password = password, url=url)

In the experiments two Fermions are loaded onto the right side, i.e. into the wire 0 and 1. In a first set of experiments there are no interactions and the two atoms are simply allowed to hop.

In [25]:
@qml.qnode(FermionDevice)
def simple_hopping(theta = 0):
    '''
    The circuit that simulates the experiments.
    
    theta ... angle of the hopping
    '''
    # load atoms
    FermionOps.Load(wires=0)
    FermionOps.Load(wires=1)
    
    # let them hop
    FermionOps.Hop(theta, wires=[0,1,2,3])

    # measure the occupation on the right side
    #obs = FermionOps.ParticleNumber(0) @ FermionOps.ParticleNumber(1)
    return [qml.expval(FermionOps.ParticleNumber(0)), qml.expval(FermionOps.ParticleNumber(1))]

In [26]:
simple_hopping(0)

{'experiment_0': {'instructions': [('load', [0], []), ('load', [1], []), ('hop', [0, 1, 2, 3], [0.0]), ('measure', [0], []), ('measure', [1], []), ('measure', [2], []), ('measure', [3], []), ('measure', [4], []), ('measure', [5], []), ('measure', [6], []), ('measure', [7], [])], 'num_wires': 1, 'shots': 500}}


tensor([1., 1.], requires_grad=True)

# And the tensor observable

In [32]:
@qml.qnode(FermionDevice)
def simple_hopping(theta = 0):
    '''
    The circuit that simulates the experiments.
    
    theta ... angle of the hopping
    '''
    # load atoms
    FermionOps.Load(wires=0)
    FermionOps.Load(wires=1)
    
    # let them hop
    FermionOps.Hop(theta, wires=[0,1,2,3])

    # measure the occupation on the right side
    return qml.sample(FermionOps.ParticleNumber(wires=[0,1]))
    #obs = FermionOps.ParticleNumber(0) @ FermionOps.ParticleNumber(1)
    #return qml.expval(obs)

In [33]:
readout = simple_hopping(np.pi)

{'experiment_0': {'instructions': [('load', [0], []), ('load', [1], []), ('hop', [0, 1, 2, 3], [1.5707963267948966]), ('measure', [0], []), ('measure', [1], []), ('measure', [2], []), ('measure', [3], []), ('measure', [4], []), ('measure', [5], []), ('measure', [6], []), ('measure', [7], [])], 'num_wires': 1, 'shots': 500}}


In [34]:
readout.shape

(500, 8)